**NBA Data Cleaning**

We first import the pandas package.


In [ ]:
import pandas as pd

We obtain the dataset link from GitHub (remote respository).  The dataset is called **Merged**  We read the excel file into a data frame.

In [ ]:
url1 = 'https://github.com/cmhamakawa/Data_Science_NBA_Project/blob/master/Datasets/Merged_NBA_Dataset(1970-2019).xlsx?raw=true'
adv_df = pd.read_excel(url1) # advanced data set is now in data frame
adv_df[1:6]

,Unnamed: 0,Rk,Player,StartYr,EndYr,SalStartYr,SalEndYr,Salary,SalaryInfl,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,1,2,Kareem Abdul-Jabbar*,1970,1971,1971,1972,NaN,NaN,C,23,MIL,82,NaN,40.1,13.0,22.5,0.577,NaN,NaN,NaN,13.0,22.5,0.577,0.577,5.7,8.3,0.690,NaN,NaN,16.0,3.3,NaN,NaN,NaN,3.2,31.7
2,2,3,Don Adams,1970,1971,1971,1972,NaN,NaN,SF,23,SDR,82,NaN,29.0,4.8,11.7,0.409,NaN,NaN,NaN,4.8,11.7,0.409,0.409,1.9,2.6,0.731,NaN,NaN,7.1,2.1,NaN,NaN,NaN,4.2,11.4
3,3,4,Rick Adelman,1970,1971,1971,1972,NaN,NaN,PG,24,POR,81,NaN,28.4,4.7,11.0,0.422,NaN,NaN,NaN,4.7,11.0,0.422,0.422,3.3,4.6,0.724,NaN,NaN,3.5,4.7,NaN,NaN,NaN,2.6,12.6
4,4,5,Lucius Allen,1970,1971,1971,1972,NaN,NaN,PG,23,MIL,61,NaN,19.0,2.9,6.5,0.447,NaN,NaN,NaN,2.9,6.5,0.447,0.447,1.3,1.8,0.700,NaN,NaN,2.5,2.6,NaN,NaN,NaN,1.8,7.1
5,5,6,Cliff Anderson,1970,1971,1971,1972,NaN,NaN,SG,26,TOT,28,NaN,7.1,0.7,2.3,0.308,NaN,NaN,NaN,0.7,2.3,0.308,0.308,1.6,2.4,0.687,NaN,NaN,1.7,0.7,NaN,NaN,NaN,1.0,3.1


We drop the first two columns (Unnamed: 0, Rk) and we add the column years of experience (years_of_exp).

In [ ]:
adv_df.drop(adv_df.columns[[0,1]], axis = 1, inplace = True)
adv_df['years_of_exp'] = adv_df.groupby('Player').cumcount()

We assume that if an NBA player is in the dataset, they have at least one year of experience.  In other words, we count years of experience at the end of a season.  So, we add 1 to the column years of experience.

In [ ]:
def add_one(x):
	return x + 1
adv_df['years_of_exp'] = adv_df['years_of_exp'].apply(add_one)

In [ ]:
adv_df[adv_df['Player'] == 'LeBron James']

,Player,StartYr,EndYr,SalStartYr,SalEndYr,Salary,SalaryInfl,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,years_of_exp
11454,LeBron James,2003,2004,2004,2005,4320360.0,5871248.0,SG,19,CLE,79,79.0,39.5,7.9,18.9,0.417,0.8,2.7,0.290,7.1,16.1,0.438,0.438,4.4,5.8,0.754,1.3,4.2,5.5,5.9,1.6,0.7,3.5,1.9,20.9,1
11908,LeBron James,2004,2005,2005,2006,4621800.0,6125892.0,SF,20,CLE,80,80.0,42.4,9.9,21.1,0.472,1.4,3.9,0.351,8.6,17.2,0.499,0.504,6.0,8.0,0.750,1.4,6.0,7.4,7.2,2.2,0.7,3.3,1.8,27.2,2
12364,LeBron James,2005,2006,2006,2007,5828090.0,7404948.0,SF,21,CLE,79,79.0,42.5,11.1,23.1,0.480,1.6,4.8,0.335,9.5,18.3,0.518,0.515,7.6,10.3,0.738,0.9,6.1,7.0,6.6,1.6,0.8,3.3,2.3,31.4,3
12822,LeBron James,2006,2007,2007,2008,13041250.0,16136130.0,SF,22,CLE,78,78.0,40.9,9.9,20.8,0.476,1.3,4.0,0.319,8.6,16.8,0.513,0.507,6.3,9.0,0.698,1.1,5.7,6.7,6.0,1.6,0.7,3.2,2.2,27.3,4
13284,LeBron James,2007,2008,2008,2009,14410581.0,16977833.0,SF,23,CLE,75,74.0,40.4,10.6,21.9,0.484,1.5,4.8,0.315,9.1,17.1,0.531,0.518,7.3,10.3,0.712,1.8,6.1,7.9,7.2,1.8,1.1,3.4,2.2,30.0,5
13731,LeBron James,2008,2009,2009,2010,15779912.0,18860203.0,SF,24,CLE,81,81.0,37.7,9.7,19.9,0.489,1.6,4.7,0.344,8.1,15.2,0.535,0.530,7.3,9.4,0.780,1.3,6.3,7.6,7.2,1.7,1.1,3.0,1.7,28.4,6
14189,LeBron James,2009,2010,2010,2011,14500000.0,17149801.0,SF,25,CLE,76,76.0,39.0,10.1,20.1,0.503,1.7,5.1,0.333,8.4,15.0,0.560,0.545,7.8,10.2,0.767,0.9,6.4,7.3,8.6,1.6,1.0,3.4,1.6,29.7,7
14635,LeBron James,2010,2011,2011,2012,16022500.0,18299290.0,SF,26,MIA,79,79.0,38.8,9.6,18.8,0.510,1.2,3.5,0.330,8.4,15.3,0.552,0.541,6.4,8.4,0.759,1.0,6.5,7.5,7.0,1.6,0.6,3.6,2.1,26.7,8
15094,LeBron James,2011,2012,2012,2013,17545000.0,19710161.0,SF,27,MIA,62,62.0,37.5,10.0,18.9,0.531,0.9,2.4,0.362,9.1,16.5,0.556,0.554,6.2,8.1,0.771,1.5,6.4,7.9,6.2,1.9,0.8,3.4,1.5,27.1,9
15551,LeBron James,2012,2013,2013,2014,19067500.0,21051220.0,PF,28,MIA,76,76.0,37.9,10.1,17.8,0.565,1.4,3.3,0.406,8.7,14.5,0.602,0.603,5.3,7.0,0.753,1.3,6.8,8.0,7.3,1.7,0.9,3.0,1.4,26.8,10


We only use players from the years 1990 and on.


In [ ]:
adv_df = adv_df[adv_df['StartYr']>=1990]
adv_df[:6]

,Player,StartYr,EndYr,SalStartYr,SalEndYr,Salary,SalaryInfl,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,years_of_exp
5782,Alaa Abdelnaby,1990,1991,1991,1992,494000.0,936409.0,PF,22,POR,43,0.0,6.7,1.3,2.7,0.474,0.0,0.0,NaN,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1
5783,Mahmoud Abdul-Rauf,1990,1991,1991,1992,2008000.0,3806296.0,PG,21,DEN,67,19.0,22.5,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1
5784,Mark Acres,1990,1991,1991,1992,547000.0,1036874.0,C,28,ORL,68,0.0,19.3,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,4
5785,Michael Adams,1990,1991,1991,1992,1309000.0,2481296.0,PG,28,DEN,66,66.0,35.5,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,6
5786,Mark Aguirre,1990,1991,1991,1992,1232000.0,2335337.0,SF,31,DET,78,13.0,25.7,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,10
5787,Danny Ainge,1990,1991,1991,1992,725000.0,1374285.0,SG,31,POR,80,0.0,21.4,4.2,8.9,0.472,1.3,3.1,0.406,2.9,5.8,0.508,0.543,1.4,1.7,0.826,0.6,2.0,2.6,3.6,0.8,0.2,1.3,2.4,11.1,10


We examine missing value counts and remove rows/columns as needed.

In [ ]:
print("Shape", adv_df.shape)
#examine missing value counts
counts = {}
for column in adv_df:
  s = adv_df[column].isna().sum()
  if s>0:
    counts[column] = s

counts

Shape (13560, 36)


{'2P%': 79,
 '3P%': 2024,
 'FG%': 49,
 'FT%': 443,
 'Salary': 3089,
 'SalaryInfl': 3089,
 'eFG%': 49}

In [ ]:
#FASTER WAY TO CHECK NAs
print("Shape", adv_df.shape)

NAs = dict(adv_df.isna().sum())
output = dict((k, v) for k, v in NAs.items() if v > 0)
output

Shape (13560, 36)


{'2P%': 79,
 '3P%': 2024,
 'FG%': 49,
 'FT%': 443,
 'Salary': 3089,
 'SalaryInfl': 3089,
 'eFG%': 49}

In [ ]:
#drop rows with remaining NAN values
adv_df.dropna(inplace=True)
adv_df.shape

(9103, 36)

We get rid of columns that are not relevant to our purpose of predicting salaries.

We load the salary cap dataset, **NBA_Salary_Cap**.

In [ ]:
# adjusting salary cap for inflation (not needed for final data set)

# !pip install cpi
# import cpi
# def inflate_column(data, column):
#     """
#     Adjust for inflation the series of values in column of the   
#     dataframe data
#     """
#     return data.apply(lambda x: cpi.inflate(x[column], 
#                       x.year), axis=1)     # converts the dollar amount in particular year to today's value
    
url2 = 'https://github.com/cmhamakawa/Data_Science_NBA_Project/blob/master/Datasets/NBA_Salary_Cap.xlsx?raw=true'
salary_cap_data = pd.read_excel(url2) # advanced data set is now in data frame

# salary cap data from 1990-2020
salary_cap_data = salary_cap_data.iloc[6:38]
salary_cap_data

,Year,SalaryCap,InflationSalaryCap
6,1990-91,11871000,22283140.0
7,1991-92,12500000,22773891.0
8,1992-93,14000000,24778161.0
9,1993-94,15175000,26176363.0
10,1994-95,15964000,26785632.0
11,1995-96,23000000,37493952.0
12,1996-97,24363000,38802016.0
13,1997-98,26900000,42195324.0
14,1998-99,30000000,46042748.0
15,1999-00,34000000,50477681.0


In [ ]:
# Christine
adv_df.columns

Index(['Player', 'StartYr', 'EndYr', 'SalStartYr', 'SalEndYr', 'Salary',
       'SalaryInfl', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'years_of_exp'],
      dtype='object')

We format the salary_cap dataset and merge it with our main dataset.



In [ ]:
# Merge salary_cap_data with advanced dataframe

def partition(val):
  return int(val.partition("-")[0])
  
salary_cap_data.Year = salary_cap_data.Year.apply(partition)

In [ ]:
salary_cap_data

,Year,SalaryCap,InflationSalaryCap
6,1990,11871000,22283140.0
7,1991,12500000,22773891.0
8,1992,14000000,24778161.0
9,1993,15175000,26176363.0
10,1994,15964000,26785632.0
11,1995,23000000,37493952.0
12,1996,24363000,38802016.0
13,1997,26900000,42195324.0
14,1998,30000000,46042748.0
15,1999,34000000,50477681.0


We mapped/matched Salary Cap Years in Salary Cap data set to the Salary Start Year in our large data set with player statistics.

In [ ]:

cap_dict = dict(zip(salary_cap_data.Year, salary_cap_data.SalaryCap))
cap_dict_adjusted = dict(zip(salary_cap_data.Year, salary_cap_data.InflationSalaryCap))
adv_df['salary_cap'] = adv_df.SalStartYr.map(cap_dict)
#adv_df['salary_cap'] = adv_df.year.map(cap_dict)
adv_df['salary_cap_adjusted'] = adv_df.SalStartYr.map(cap_dict_adjusted)

In [ ]:
# ADHVAITH
adv_df.head()

,Player,StartYr,EndYr,SalStartYr,SalEndYr,Salary,SalaryInfl,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,years_of_exp,salary_cap,salary_cap_adjusted
5783,Mahmoud Abdul-Rauf,1990,1991,1991,1992,2008000.0,3806296.0,PG,21,DEN,67,19.0,22.5,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1,12500000,22773891.0
5784,Mark Acres,1990,1991,1991,1992,547000.0,1036874.0,C,28,ORL,68,0.0,19.3,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,4,12500000,22773891.0
5785,Michael Adams,1990,1991,1991,1992,1309000.0,2481296.0,PG,28,DEN,66,66.0,35.5,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,6,12500000,22773891.0
5786,Mark Aguirre,1990,1991,1991,1992,1232000.0,2335337.0,SF,31,DET,78,13.0,25.7,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,10,12500000,22773891.0
5787,Danny Ainge,1990,1991,1991,1992,725000.0,1374285.0,SG,31,POR,80,0.0,21.4,4.2,8.9,0.472,1.3,3.1,0.406,2.9,5.8,0.508,0.543,1.4,1.7,0.826,0.6,2.0,2.6,3.6,0.8,0.2,1.3,2.4,11.1,10,12500000,22773891.0


In [ ]:
adv_df.shape

(9103, 38)

We make our predictor column, Percent of Salary Cap (Per_of_Salary_Cap) by dividing the player's salary for that year by the salary_cap for that same year.

In [ ]:
# normalizing salaries by dividing salary by salary cap of that year
adv_df['Per_of_Salary_Cap'] = adv_df.Salary / adv_df.salary_cap
adv_df.head()

,Player,StartYr,EndYr,SalStartYr,SalEndYr,Salary,SalaryInfl,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,years_of_exp,salary_cap,salary_cap_adjusted,Per_of_Salary_Cap
5783,Mahmoud Abdul-Rauf,1990,1991,1991,1992,2008000.0,3806296.0,PG,21,DEN,67,19.0,22.5,6.2,15.1,0.413,0.4,1.5,0.240,5.9,13.6,0.432,0.425,1.3,1.5,0.857,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1,12500000,22773891.0,0.16064
5784,Mark Acres,1990,1991,1991,1992,547000.0,1036874.0,C,28,ORL,68,0.0,19.3,1.6,3.1,0.509,0.0,0.0,0.333,1.6,3.1,0.512,0.512,1.0,1.5,0.653,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,4,12500000,22773891.0,0.04376
5785,Michael Adams,1990,1991,1991,1992,1309000.0,2481296.0,PG,28,DEN,66,66.0,35.5,8.5,21.5,0.394,2.5,8.5,0.296,6.0,13.0,0.459,0.453,7.0,8.0,0.879,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,6,12500000,22773891.0,0.10472
5786,Mark Aguirre,1990,1991,1991,1992,1232000.0,2335337.0,SF,31,DET,78,13.0,25.7,5.4,11.7,0.462,0.3,1.0,0.308,5.1,10.7,0.477,0.475,3.1,4.1,0.757,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,10,12500000,22773891.0,0.09856
5787,Danny Ainge,1990,1991,1991,1992,725000.0,1374285.0,SG,31,POR,80,0.0,21.4,4.2,8.9,0.472,1.3,3.1,0.406,2.9,5.8,0.508,0.543,1.4,1.7,0.826,0.6,2.0,2.6,3.6,0.8,0.2,1.3,2.4,11.1,10,12500000,22773891.0,0.05800


In [ ]:
list(adv_df.columns)

['SalStartYr',
 'Salary',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'years_of_exp',
 'salary_cap',
 'Per_of_Salary_Cap']

We download our preprocessed dataset.

In [ ]:
from google.colab import files
adv_df.to_csv('preprocessed_nba_dataset(1990-2020).csv') 
files.download('preprocessed_nba_dataset(1990-2020).csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>